In [2]:
import os
import sys

sys.path.insert(0, '../')
sys.path.insert(0, '../utils/')

import utils.data as dat
import utils.models as mdl
import utils.variables as vrb
import utils.io_manager as iom

import pandas as pd

In [3]:
df_original = dat.get_data_model()

In [4]:
prediction_interval = 1
set_series = {}

columns_1 = [dat.TARGET]
set_series[1] = mdl.get_timeseries_train_valid_test_sets(df_original, columns_1, prediction_interval)

columns_2 = dat.get_colums_original_features_2()
set_series[2] = mdl.get_timeseries_train_valid_test_sets(df_original, columns_2, prediction_interval)

columns_5 = dat.get_columns_sizes_features_5()
set_series[5] = mdl.get_timeseries_train_valid_test_sets(df_original, columns_5, prediction_interval)

columns_11 = dat.get_columns_all_features_11(df_original)
set_series[11] = mdl.get_timeseries_train_valid_test_sets(df_original, columns_11, prediction_interval)


In [7]:
result_serie = {"file":[], "set":[], "loss":[], "MAE":[], "RMSE":[]}
directory = ""
file_list = os.listdir(vrb.SAVED_MODELS_DIRECTORY + directory) 
for idx, file in enumerate(file_list):
    filename = iom.get_model_name(file).replace(file, directory + file).replace(".pkl","")
    print(f"{filename} -> Check: {filename.find("/model_") > 0}")
    
    if filename.find("/model_") > 0:
        features_str = file[file.index("var"): file.index("_out")]
        features_num = int(features_str.replace("var",""))
        print(f"* ({idx+ 1}/{len(file_list)}) {file} -> {features_num} columns")

        series_eval = {}
        series_eval["Valid set"] = set_series[features_num][1]
        series_eval["Test set"] = set_series[features_num][2]

        model = iom.read_model(filename)
        print(f"\t> Model ready: {model}")

        for k in series_eval.keys():
            print(f"\tSet: {k}")
            set_loss, set_mae, set_rmse = mdl.model_evaluate(model, series_eval[k])
            print(f"\t   - LOSS: {set_loss}")
            print(f"\t   - MAE: {set_mae}")
            print(f"\t   - RMSE: {set_rmse}")

            result_serie["file"].append(file)
            result_serie["set"].append(k)
            result_serie["loss"].append(set_loss)
            result_serie["MAE"].append(set_mae)
            result_serie["RMSE"].append(set_rmse)


../models/model_1_Dense1_multivar11_out1 -> Check: True
* (1/21) model_1_Dense1_multivar11_out1.pkl -> 11 columns
	> Model ready: <Sequential name=sequential_10, built=True>
	Set: Valid set
294/294 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0026 - mae: 0.0662 - root_mean_squared_error: 0.0701
	   - LOSS: 0.0024886003229767084
	   - MAE: 0.0651654377579689
	   - RMSE: 0.07054927945137024
	Set: Test set
302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0015 - mae: 0.0531 - root_mean_squared_error: 0.0542
	   - LOSS: 0.0015507661737501621
	   - MAE: 0.053666457533836365
	   - RMSE: 0.05569140613079071
../models/model_1_Dense1_multivar2_out1 -> Check: True
* (2/21) model_1_Dense1_multivar2_out1.pkl -> 2 columns
	> Model ready: <Sequential name=sequential, built=True>
	Set: Valid set
294/294 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0043 - mae: 0.0856 - root_mean_squared_error: 0.0907
	   - LOSS: 0.004151057451963425
	   - MAE: 0.08398278057575226
	   - RMSE: 0.09111594408750534
	Set: Tes

In [8]:
df = pd.DataFrame(result_serie)

In [9]:
df.to_csv(vrb.TEST_RESULTS_PATH)

In [10]:
df.sort_values("MAE", ascending=True)

,file,set,loss,MAE,RMSE
39,model_5_GRU32_univar1_out1.pkl,Test set,6.098299e-07,0.000728,0.001104
23,model_3_RNN32_univar1_out1.pkl,Test set,6.220657e-07,0.000756,0.001115
35,model_5_GRU32_multivar2_out1.pkl,Test set,6.626872e-07,0.000804,0.001151
33,model_5_GRU32_multivar11_out1.pkl,Test set,6.501631e-07,0.000807,0.001140
37,model_5_GRU32_multivar5_out1.pkl,Test set,7.580566e-07,0.000859,0.001231
29,model_4_LSTM32_multivar5_out1.pkl,Test set,9.115304e-07,0.000917,0.001350
31,model_4_LSTM32_univar1_out1.pkl,Test set,9.519916e-07,0.000928,0.001380
15,model_2_RNN1_univar1_out1.pkl,Test set,9.774875e-07,0.000972,0.001398
17,model_3_RNN32_multivar11_out1.pkl,Test set,1.073064e-06,0.001001,0.001465
27,model_4_LSTM32_multivar2_out1.pkl,Test set,1.220592e-06,0.001096,0.001562


In [11]:
df[df["file"]=="model_5_GRU32_univar1_out1.pkl"]

,file,set,loss,MAE,RMSE
38,model_5_GRU32_univar1_out1.pkl,Valid set,1.344191e-06,0.001124,0.001640
39,model_5_GRU32_univar1_out1.pkl,Test set,6.098299e-07,0.000728,0.001104
